In [43]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [44]:
df_quantity = pd.read_csv('ratnagiri_mango_verified_2012_2024.csv')
df_daily_weather = pd.read_csv('final_daily_weather.csv', parse_dates=["date"])
df_hourly_weather = pd.read_csv('final_hourly_weather.csv', parse_dates=["date"])

In [45]:
def create_weather_features(daily_df, hourly_df=None):
    """
    Generate weather features for mango production modeling.
    daily_df must have:
      - date (datetime64)
      - any of temperature_2m_max, temperature_2m_min, temperature_2m_mean
      - any of precipitation_sum, rain_sum, rainfall
    hourly_df (optional) must have:
      - date (datetime64)
      - relative_humidity_2m or humidity
    """

    # Ensure date columns are datetime
    daily = daily_df.copy()
    daily['date'] = pd.to_datetime(daily['date'])
    daily['year'] = daily['date'].dt.year
    daily['month'] = daily['date'].dt.month

    if hourly_df is not None:
        hourly = hourly_df.copy()
        hourly['date'] = pd.to_datetime(hourly['date'])
        hourly['year'] = hourly['date'].dt.year
        hourly['month'] = hourly['date'].dt.month
    else:
        hourly = None

    # Define critical periods
    periods = {
        'flowering': [11,12,1,2],
        'fruit_dev': [3,4,5],
        'harvest':   [4,5,6]
    }

    def stat(df, col_options, year, months, func, **kwargs):
        """Select column from col_options present in df, filter by year/months, and apply func."""
        cols = [c for c in col_options if c in df.columns]
        if not cols:
            return np.nan
        col = cols[0]
        # handle flowering crossing year boundary
        if set(months) & {11,12}:
            prev = df[(df['year']==year-1)&(df['month'].isin([m for m in months if m>10]))]
            curr = df[(df['year']==year)&(df['month'].isin([m for m in months if m<11]))]
            data = pd.concat([prev,curr])
        else:
            data = df[(df['year']==year)&(df['month'].isin(months))]
        if data.empty:
            return np.nan
        series = data[col].dropna()
        if series.empty:
            return np.nan
        if func=='mean':
            return series.mean()
        if func=='max':
            return series.max()
        if func=='min':
            return series.min()
        if func=='sum':
            return series.sum()
        if func=='std':
            return series.std()
        if func=='count_above':
            return (series>kwargs['threshold']).sum()
        if func=='count_below':
            return (series<kwargs['threshold']).sum()
        return np.nan

    def max_dry(df, year, months):
        """Max consecutive dry days where rainfall ≤ 1 mm."""
        cols = [c for c in ['rain_sum','rainfall'] if c in df.columns]
        if not cols: return np.nan
        col = cols[0]
        if set(months)&{11,12}:
            prev = df[(df['year']==year-1)&(df['month'].isin([m for m in months if m>10]))]
            curr = df[(df['year']==year)&(df['month'].isin([m for m in months if m<11]))]
            data = pd.concat([prev,curr])
        else:
            data = df[(df['year']==year)&(df['month'].isin(months))]
        if data.empty:
            return np.nan
        dry = (data[col] <= 1).astype(int).values
        maxc = cur = 0
        for d in dry:
            if d: cur+=1; maxc=max(maxc,cur)
            else: cur=0
        return maxc

    features = {}
    for year in range(2012, 2025):
        feats = {}
        for name, months in periods.items():
            # Temperature stats
            feats[f'{name}_avg_temp']  = stat(daily, ['temperature_2m_mean','temperature_2m_max','temperature_2m_min'], year, months, 'mean')
            feats[f'{name}_min_temp']  = stat(daily, ['temperature_2m_min','temperature_2m_mean'], year, months, 'min')
            feats[f'{name}_max_temp']  = stat(daily, ['temperature_2m_max','temperature_2m_mean'], year, months, 'max')
            feats[f'{name}_std_temp']  = stat(daily, ['temperature_2m_mean'], year, months, 'std')
            feats[f'{name}_heat_days']= stat(daily, ['temperature_2m_max'], year, months, 'count_above', threshold=35)
            feats[f'{name}_cold_days']= stat(daily, ['temperature_2m_min'], year, months, 'count_below', threshold=20)
            # Rainfall stats
            feats[f'{name}_total_rain']= stat(daily, ['rain_sum','rainfall'], year, months, 'sum')
            feats[f'{name}_avg_rain']  = stat(daily, ['rain_sum','rainfall'], year, months, 'mean')
            feats[f'{name}_rainy_days']= stat(daily, ['rain_sum','rainfall'], year, months, 'count_above', threshold=1)
            feats[f'{name}_heavy_rain']= stat(daily, ['rain_sum','rainfall'], year, months, 'count_above', threshold=50)
            feats[f'{name}_dry_spell'] = max_dry(daily, year, months)
            # Humidity stats (hourly)
            if hourly_df is not None:
                feats[f'{name}_avg_hum'] = stat(hourly_df, ['relative_humidity_2m','humidity'], year, months, 'mean')
                feats[f'{name}_high_hum_hrs'] = stat(hourly_df, ['relative_humidity_2m'], year, months, 'count_above', threshold=80)
        # Annual indices
        feats['annual_gdd'] = feats['flowering_avg_temp']*len(daily[daily['year']==year]) if not np.isnan(feats['flowering_avg_temp']) else np.nan
        feats['annual_total_rain'] = stat(daily, ['precipitation_sum','rain_sum','rainfall'], year, list(range(1,13)), 'sum')
        # Interaction example
        if not np.isnan(feats['flowering_avg_temp']) and not np.isnan(feats['flowering_total_rain']):
            feats['flowering_temp_rain_int'] = feats['flowering_avg_temp'] * feats['flowering_total_rain'] / 100
        features[year] = feats

    return pd.DataFrame.from_dict(features, orient='index')

In [46]:
weather_feats = create_weather_features(df_daily_weather)
weather_feats.head(10)

,flowering_avg_temp,flowering_min_temp,flowering_max_temp,flowering_std_temp,flowering_heat_days,flowering_cold_days,flowering_total_rain,flowering_avg_rain,flowering_rainy_days,flowering_heavy_rain,...,harvest_heat_days,harvest_cold_days,harvest_total_rain,harvest_avg_rain,harvest_rainy_days,harvest_heavy_rain,harvest_dry_spell,annual_gdd,annual_total_rain,flowering_temp_rain_int
2012,24.797034,16.262000,37.312,1.602049,3,36,0.000000,0.000000,0,0,...,3,0,538.999996,5.923077,30,4,43,9075.714590,2108.799996,0.000000
2013,26.119187,16.562000,36.062,1.291563,2,45,6.400000,0.053333,2,0,...,4,0,693.999987,7.626373,35,3,55,9533.503425,2574.600001,1.671628
2014,25.773285,16.712000,34.462,1.401137,0,41,3.400000,0.028333,1,0,...,18,0,178.799997,1.964835,33,0,20,9407.248934,1975.999990,0.876292
2015,25.672902,16.412000,36.362,1.332107,1,41,57.800000,0.481667,5,0,...,10,0,573.899989,6.306593,30,2,27,9370.609330,1571.499976,14.838938
2016,26.510192,18.012001,35.862,1.407750,3,30,36.100000,0.298347,7,0,...,19,0,722.099998,7.935165,37,4,54,9702.730230,2321.599985,9.570179
2017,26.160576,17.862000,36.421,1.430551,12,36,3.300000,0.027500,1,0,...,0,0,896.300016,9.849451,37,4,43,9548.610383,3213.600043,0.863299
2018,26.059507,18.821001,36.471,1.374527,3,20,12.600000,0.105000,2,0,...,1,0,1007.699972,11.073626,33,5,41,9511.720116,3216.599962,3.283498
2019,25.901104,14.321000,35.271,1.714787,1,24,22.000001,0.183333,6,0,...,0,0,605.699990,6.656044,24,3,48,9453.903127,4183.300010,5.698243
2020,26.652319,17.321001,36.771,1.744848,3,11,18.800000,0.155372,5,0,...,3,0,875.199985,9.617582,36,5,15,9754.748836,4183.699987,5.010636
2021,26.936972,19.821001,37.321,1.072825,2,5,15.800000,0.131667,6,0,...,1,0,1069.500027,11.752748,45,6,15,9831.994813,3592.900045,4.256042


In [47]:
# Check what weather features were created
print("Weather features shape:", weather_feats.shape)
print("Years covered:", weather_feats.index.tolist())
print("\nFirst few columns:", weather_feats.columns[:10].tolist())
print("\nSample of data:")
print(weather_feats.head(3))

# Check for missing values
print("\nMissing values by feature:")
missing = weather_feats.isnull().sum()
print(missing[missing > 0].sort_values(ascending=False))

Weather features shape: (13, 36)
Years covered: [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]

First few columns: ['flowering_avg_temp', 'flowering_min_temp', 'flowering_max_temp', 'flowering_std_temp', 'flowering_heat_days', 'flowering_cold_days', 'flowering_total_rain', 'flowering_avg_rain', 'flowering_rainy_days', 'flowering_heavy_rain']

Sample of data:
      flowering_avg_temp  flowering_min_temp  flowering_max_temp  \
2012           24.797034              16.262              37.312   
2013           26.119187              16.562              36.062   
2014           25.773285              16.712              34.462   

      flowering_std_temp  flowering_heat_days  flowering_cold_days  \
2012            1.602049                    3                   36   
2013            1.291563                    2                   45   
2014            1.401137                    0                   41   

      flowering_total_rain  flowering_avg_rain  flowe

In [48]:
# Merge weather features with production data
# Assuming your production DataFrame is df_quantity with columns 'Year' and 'Production_MT'

# Align the data
production_clean = df_quantity[df_quantity['Production_MT'].notna()].copy()
production_clean = production_clean.set_index('Year')['Production_MT']

print("Production data years:", production_clean.index.tolist())
print("Weather data years:", weather_feats.index.tolist())

# Find common years
common_years = sorted(set(production_clean.index) & set(weather_feats.index))
print("Common years for modeling:", common_years)

# Create final modeling dataset
X = weather_feats.loc[common_years]
y = production_clean.loc[common_years]

print(f"\nFinal dataset: {len(common_years)} years, {X.shape[1]} features")
print("Target variable (Production):")
print(y)

Production data years: [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
Weather data years: [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
Common years for modeling: [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]

Final dataset: 13 years, 36 features
Target variable (Production):
Year
2012    117664
2013    119500
2014    121200
2015    124800
2016    126500
2017    118200
2018    115800
2019    112400
2020    108900
2021    105600
2022    102300
2023    109800
2024    114200
Name: Production_MT, dtype: int64


In [49]:
# Split the data (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set: {X_train.shape[0]} years")
print(f"Test set: {X_test.shape[0]} years")

Training set: 10 years
Test set: 3 years


In [50]:
# Scale the features (important for some models)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [51]:
# Test multiple models
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=0.1),
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)
}

results = {}

for name, model in models.items():
    print(f"\n--- {name} ---")
    
    # Use scaled data for linear models, original for Random Forest
    if 'Forest' in name:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        X_cv = X
    else:
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled) 
        X_cv = scaler.fit_transform(X)
    
    # Calculate metrics
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    
    # Cross-validation score
    cv_scores = cross_val_score(model, X_cv, y, cv=5, scoring='r2')
    
    results[name] = {
        'R²': r2,
        'RMSE': rmse,
        'MAE': mae,
        'CV_R²_mean': cv_scores.mean(),
        'CV_R²_std': cv_scores.std()
    }
    
    print(f"R²: {r2:.3f}")
    print(f"RMSE: {rmse:.2f} MT")
    print(f"MAE: {mae:.2f} MT")
    print(f"Cross-val R²: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")

# Summary table
print("\n" + "="*60)
print("MODEL COMPARISON SUMMARY")
print("="*60)
import pandas as pd
results_df = pd.DataFrame(results).T
print(results_df.round(3))


--- Linear Regression ---
R²: -0.255
RMSE: 5601.16 MT
MAE: 5440.40 MT
Cross-val R²: -14.239 ± 23.474

--- Ridge Regression ---
R²: -0.431
RMSE: 5982.21 MT
MAE: 5707.84 MT
Cross-val R²: -14.687 ± 24.364

--- Lasso Regression ---
R²: -4.730
RMSE: 11969.70 MT
MAE: 9920.92 MT
Cross-val R²: -28.323 ± 39.829

--- Random Forest ---
R²: -0.743
RMSE: 6600.75 MT
MAE: 5872.33 MT
Cross-val R²: -10.441 ± 15.661

MODEL COMPARISON SUMMARY
                      R²       RMSE       MAE  CV_R²_mean  CV_R²_std
Linear Regression -0.255   5601.162  5440.397     -14.239     23.474
Ridge Regression  -0.431   5982.208  5707.840     -14.687     24.364
Lasso Regression  -4.730  11969.702  9920.921     -28.323     39.829
Random Forest     -0.743   6600.753  5872.333     -10.441     15.661


In [52]:
# Use the weather features that correlate with production
# Based on your findings: temperature_max and humidity (positive), rain (negative)

# Select features that align with your correlation findings
targeted_features = []

# Temperature features (positive correlation with production)
temp_features = [col for col in weather_feats.columns if 'avg_temp' in col or 'max_temp' in col]
targeted_features.extend(temp_features)

# Humidity features (positive correlation)
humidity_features = [col for col in weather_feats.columns if 'avg_hum' in col]
targeted_features.extend(humidity_features)

# Rain features (negative correlation with production)
rain_features = [col for col in weather_feats.columns if 'total_rain' in col or 'heavy_rain' in col]
targeted_features.extend(rain_features)

# Heat stress (likely negative impact)
heat_features = [col for col in weather_feats.columns if 'heat_days' in col]
targeted_features.extend(heat_features)

# Remove duplicates and ensure features exist
targeted_features = list(set(targeted_features))
targeted_features = [f for f in targeted_features if f in weather_feats.columns]

print("Selected features based on correlation insights:")
for i, feat in enumerate(targeted_features, 1):
    print(f"{i:2d}. {feat}")

# Create the targeted dataset
X_targeted = weather_feats[targeted_features].loc[common_years]
y_targeted = production_clean.loc[common_years]

print(f"\nTargeted dataset: {len(common_years)} years, {len(targeted_features)} features")

Selected features based on correlation insights:
 1. fruit_dev_total_rain
 2. harvest_total_rain
 3. flowering_total_rain
 4. flowering_avg_temp
 5. harvest_heavy_rain
 6. harvest_max_temp
 7. flowering_heavy_rain
 8. annual_total_rain
 9. fruit_dev_avg_temp
10. fruit_dev_heavy_rain
11. fruit_dev_heat_days
12. harvest_heat_days
13. flowering_heat_days
14. fruit_dev_max_temp
15. flowering_max_temp
16. harvest_avg_temp

Targeted dataset: 13 years, 16 features


In [53]:
# First, let's look at our data to understand the issue
print("Dataset info:")
print(f"Samples: {len(X)} years")
print(f"Features: {X.shape[1]}")
print(f"Features-to-samples ratio: {X.shape[1]/len(X):.1f}")
print("\nProduction values:")
print(y)
print(f"Production range: {y.min():.1f} - {y.max():.1f} MT")
print(f"Production std: {y.std():.1f} MT")

# Check for multicollinearity - highly correlated features
correlation_matrix = X.corr()
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:  # High correlation threshold
            high_corr_pairs.append((correlation_matrix.columns[i], 
                                   correlation_matrix.columns[j], 
                                   correlation_matrix.iloc[i, j]))

print(f"\nHighly correlated feature pairs (>0.8): {len(high_corr_pairs)}")
for feat1, feat2, corr in high_corr_pairs[:10]:  # Show first 10
    print(f"{feat1} <-> {feat2}: {corr:.3f}")


Dataset info:
Samples: 13 years
Features: 36
Features-to-samples ratio: 2.8

Production values:
Year
2012    117664
2013    119500
2014    121200
2015    124800
2016    126500
2017    118200
2018    115800
2019    112400
2020    108900
2021    105600
2022    102300
2023    109800
2024    114200
Name: Production_MT, dtype: int64
Production range: 102300.0 - 126500.0 MT
Production std: 7222.6 MT

Highly correlated feature pairs (>0.8): 20
flowering_avg_temp <-> annual_gdd: 0.999
flowering_total_rain <-> flowering_avg_rain: 1.000
flowering_total_rain <-> flowering_rainy_days: 0.936
flowering_total_rain <-> flowering_temp_rain_int: 0.999
flowering_avg_rain <-> flowering_rainy_days: 0.936
flowering_avg_rain <-> flowering_temp_rain_int: 0.999
flowering_rainy_days <-> flowering_temp_rain_int: 0.931
fruit_dev_avg_temp <-> fruit_dev_heat_days: 0.809
fruit_dev_min_temp <-> fruit_dev_cold_days: -0.848
fruit_dev_heat_days <-> harvest_avg_temp: 0.805


In [54]:
# Drop multicollinear features
drop_cols = [
    'annual_gdd',            # duplicate of flowering_avg_temp
    'flowering_avg_rain',    # duplicate of flowering_total_rain
    'flowering_rainy_days',
    'flowering_temp_rain_int'
]
X_mc = X.drop(columns=drop_cols)
print("Reduced multicollinearity:", X_mc.shape[1], "features remaining")

Reduced multicollinearity: 32 features remaining


In [57]:
# Two features
X2 = X[['flowering_avg_temp','flowering_total_rain']]

loo = LeaveOneOut()
model = Ridge(alpha=5.0)

cv_r2   = cross_val_score(model, X2, y, cv=loo, scoring='r2')
cv_rmse = -cross_val_score(model, X2, y, cv=loo, scoring='neg_root_mean_squared_error')

print(f"Two-feature Ridge R²:   {cv_r2.mean():.3f} ± {cv_r2.std():.3f}")
print(f"Two-feature Ridge RMSE: {cv_rmse.mean():.1f} MT ± {cv_rmse.std():.1f}")

Two-feature Ridge R²:   nan ± nan
Two-feature Ridge RMSE: 7376.7 MT ± 6434.6


/home/loijun/mango-price-predict/venv/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1283: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/loijun/mango-price-predict/venv/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1283: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/loijun/mango-price-predict/venv/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1283: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/loijun/mango-price-predict/venv/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1283: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/loijun/mango-price-predict/venv/lib/python3.13/site-packages/sklearn/m

In [60]:
for feat in ['flowering_avg_temp','flowering_total_rain']:
    Xf = X[[feat]]
    cv_r2   = cross_val_score(Ridge(alpha=5), Xf, y, cv=loo, scoring='r2')
    cv_rmse = -cross_val_score(Ridge(alpha=5), Xf, y, cv=loo, scoring='neg_root_mean_squared_error')
    print(f"{feat} alone — R²: {cv_r2.mean():.3f}, RMSE: {cv_rmse.mean():.1f} MT")

flowering_avg_temp alone — R²: nan, RMSE: 6186.3 MT
flowering_total_rain alone — R²: nan, RMSE: 6689.8 MT


/home/loijun/mango-price-predict/venv/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1283: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/loijun/mango-price-predict/venv/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1283: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/loijun/mango-price-predict/venv/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1283: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/loijun/mango-price-predict/venv/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1283: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/loijun/mango-price-predict/venv/lib/python3.13/site-packages/sklearn/m